 - [Introduction](#section-one)
 - [Data exploration](#section-two)
 - [The model](#section-three)
 - [What went wrong/right](#section-four)


## Please upvote if you found this interesting, and comment if you have any improvements :)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

<a name="section-one"></a>
## Introduction

I am very much a begginer when it comes to data science, but I am really enjoying the learning process - so far anyway. I have been through many tutorials and I really need to start doing some competitions so I can actually learn. So here I am!

Often I find sarting the hardest part; this is especially true with deep learning I am finding. Classification and regression competitions seem much easier. 

So, here I am giving it a go in what I hope is a good starting point for a deep learning competition.

### Progression
1. achieved 0.95120
2. added learning rate annealer and increased epochs by 10, achieved 0.97380
3. Changed from Sequential to functional - no change to score
    - Decreased dropout slightly.
    - Increased image zoom and height and width_shift_range from 0.1 to 0.25
4. added another dense layer and another convolutional layer, reduced the kernel size in the 1st layer. Also increased the test size by 10% - achieved 0.97620
5. Completely changed the model to this specification https://www.kaggle.com/anshumandec94/6-layer-conv-nn-using-adam. I still have no idea where people get the ideas for the design of these models, if you know please comment below. achieved 0.97780
6. changed optimizer from RMSprop to Adam 0.975
7. Edited Adam optimizer; increased epochs; changed learning rate to 0.33 from 0.5; reduced test size. 
8. incorporated extra val set into training
9. Increased epochs from 50 to 65
10. changed learning rate and batch size
11. removed digits dataset from training.
12. added model checkpoint
13. changed batch size to 100
#### Completely changed my approach

In [ ]:
#Import Packages

import random
import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as mpatches

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from math import sqrt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input, BatchNormalization
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint

In [ ]:
keras.backend.clear_session()

<a name="section-two"></a>
## Data Exploration

In [ ]:
train = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')
#val_set = pd.read_csv("../input/Kannada-MNIST/Dig-MNIST.csv")
#train = pd.concat([train, val_set], axis=0)
y_train = train['label']
X = train.drop(['label'],axis=1)
del train
Id = test['id'] # save this for the submission
test = test.drop(['id'],axis=1) #

In [ ]:
X.shape

In [ ]:
test.shape

Much fewer samples in the test set.

In [ ]:
plt.imshow(X.values[3].reshape(28,28))

Interesting, I cannot distinguish what this is, I hope a machine can.

In [ ]:
# code for this plot taken from https://www.kaggle.com/josephvm/kannada-with-pytorch

fig, ax = plt.subplots(nrows=10, ncols=10, figsize=(15,15))

# I know these for loops look weird, but this way num_i is only computed once for each class
for i in range(10): # Column by column
    num_i = X[y_train == i]
    ax[0][i].set_title(i)
    for j in range(10): # Row by row
        ax[j][i].axis('off')
        ax[j][i].imshow(num_i.iloc[j, :].to_numpy().astype(np.uint8).reshape(28, 28))


In hindsight, if I were developing a number system, I would make sure that the symbols are as different as possible - 3 and 7 are basically the same, as are 6 and 9! Equations in kannada are probably quite interesting

In [ ]:
import seaborn as sns
lab, val = np.unique(y_train,return_counts=True)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,7))
sns.barplot(lab,val)
plt.title('Equal number of samples in all classes in the training data')


Hopefully the test set is as equally spread.

In [ ]:
first_1000 = y_train[:10]
np.unique(first_1000, return_counts=True)

So the numbers repeat every 10, that is usefull! let's see if the number of pixels that are non '0' are informative to the label? I suspect some numbers require more 'ink' than others

In [ ]:
labels =[]
frequencies = []


for i in range(len(y_train)):
    lab, freq = str(y_train[i]), len([n for n in X.values[i] if n > 0])
    labels.append(lab)
    frequencies.append(freq)
    
    
data = {'Labels':labels, 'Frequencies':frequencies}

df = pd.DataFrame(data)

In [ ]:
# mean number of pixels per label
df.groupby('Labels').mean()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
sns.boxplot(x = 'Labels', y = 'Frequencies', data = df)


It looks like some numbers generally require more pixels e.g 8, but the variance is high as peoples handwriting varies quite a lot I suspect. Oh, well it was worth a try

In [ ]:
# apply threshold
image = X.values[3].reshape(28,28)
thresh = threshold_otsu(image)
bw = closing(image > thresh, square(3))

# label image regions
label_image = label(bw)
image_label_overlay = label2rgb(label_image, image=image)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for region in regionprops(label_image):
    # take regions with large enough areas
    if region.area >= 30:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='green', linewidth=2)
        ax.add_patch(rect)
plt.text(1,1, f'Width: {maxc-minc} Height: {maxr -minr} Diagonal: {round(sqrt((((maxc-minc)**2) + (maxr-minr)**2)),2)}', color = 'w')
ax.set_axis_off()
plt.tight_layout()

In [ ]:
def measurements(images):
    
    widths = []
    heights = []
    diags = []
    
    for i in range(len(images)):
        # apply threshold
        image = images[i].reshape(28,28)
        thresh = threshold_otsu(image)
        bw = closing(image > thresh, square(3))

        # label image regions
        label_image = label(bw)
        image_label_overlay = label2rgb(label_image, image=image)

        for region in regionprops(label_image):
            # take regions with large enough areas
            if region.area >= 30:
                # draw rectangle around segmented coins
                minr, minc, maxr, maxc = region.bbox
                
        widths.append(maxc-minc)
        heights.append(maxr -minr)
        diags.append(sqrt((((maxc-minc)**2) + (maxr-minr)**2)))
                     
    return widths, heights, diags
                     


widths, heights, diags = measurements(X.values)
                     
data = {'Labels':labels, 'diagonals':diags, 'widths':widths, 'heights':heights, 'Area':frequencies} #area is not actually area, but the number of pixels

df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
sns.boxplot(x = 'Labels', y = 'diagonals', data = df)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
sns.boxplot(x = 'Labels', y = 'heights', data = df)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
sns.boxplot(x = 'Labels', y = 'widths', data = df)



So, what have I learnt? Well, the height is very similar for each digit. This is strange to me as I would have though handwriting would vary alot more than that. Perhaps there is a logic error in my code? This is also true in the 'digits' dataset.

the width of the digit is much more variable but still, its not a huge difference.

Let's see if a decision tree can tell the difference given these data


In [ ]:


from sklearn.ensemble import RandomForestClassifier
test_y = df['Labels']

test_x = df
test_x.drop('Labels', axis =1, inplace = True)



In [ ]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(test_x, test_y, test_size=0.1, random_state=1337)

In [ ]:
clf_rfc = RandomForestClassifier(n_estimators = 100)
clf_rfc.fit(X_train1,y_train1)



from sklearn.metrics import accuracy_score

accuracy_score(clf_rfc.predict(X_val1), y_val1)



Better than that of the mnist data!, but still terrible. To see the comparison look here https://www.kaggle.com/jakelj/mnist-digits-novice-to-master

<a name="section-three"></a>
## Building the model

In [ ]:
# re-shaping the data so that keras can use it, this is something that trips me up every time

X = X.values.reshape(X.shape[0], 28, 28,1)
test = test.values.reshape(test.shape[0], 28, 28,1)

# Scale the data
X = X / 255.0
test = test / 255.0

In [ ]:
# This modifies some images slightly, I have seen this in a few tutorials and it usually makes the model more accurate. As a beginner, it goes without saying I don't fully understand all the parameters

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.25, # Randomly zoom image 
        width_shift_range=0.25,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.25,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images



valid_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.25, # Randomly zoom image 
        width_shift_range=0.25,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.25,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images) 

In [ ]:
y_train = to_categorical(y_train,num_classes=10) # the labels need to be one-hot encoded, this is something else I usually forget

In [ ]:
# same as https://www.kaggle.com/bustam/cnn-in-keras-for-kannada-digits

learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', 
                                            patience=3, 
                                            verbose=10, 
                                            factor=0.33, 
                                            min_lr=0.00001)
checkpoint=ModelCheckpoint('bestweights.md5', monitor='accuracy', verbose=1, save_best_only=True)


I have decided that I will try and make as simple model as possible, as it is all well and good modifying someone elses model, but you do not learn as much that way, so from now on I will aim for as few layers as possible

In [ ]:
epochs = 40
initial_learningrate=2e-3
batch_size = 100

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size=0.1, random_state=1)

In [ ]:


def build_model1(input_shape=(28, 28, 1), classes = 10):
    
    activation = 'relu'
    padding = 'same'
    gamma_initializer = 'uniform'
    
    input_layer = Input(shape=input_shape)
    
    hidden=Conv2D(32, (11,11), strides=1, padding =padding, activation = activation, name = 'conv1')(input_layer)
    hidden=BatchNormalization()(hidden)
    hidden=Conv2D(32, (11,11), strides=1, padding =padding, activation = activation, name = 'conv2')(hidden)
    hidden=BatchNormalization()(hidden)
    hidden=Conv2D(32, (5,5), strides=1, padding =padding, activation = activation, name = 'conv3')(hidden)
    hidden=BatchNormalization()(hidden)
    hidden=MaxPool2D()(hidden)
    hidden=Dropout(0.35)(hidden)
    
    hidden=Conv2D(64, (3,3), strides=1, padding =padding, activation = activation, name = 'conv4')(input_layer)
    hidden=BatchNormalization()(hidden)
    hidden=Conv2D(64, (3,3), strides=1, padding =padding, activation = activation, name = 'conv5')(hidden)
    hidden=BatchNormalization()(hidden)
    hidden=Conv2D(64, (5,5), strides=1, padding =padding, activation = activation, name = 'conv6')(hidden)
    hidden=BatchNormalization()(hidden)
    hidden=MaxPool2D()(hidden)
    hidden=Dropout(0.3)(hidden)
    

    hidden=Flatten()(hidden)
    hidden=Dense(250,activation = activation, name="Dense1")(hidden)
    hidden=Dropout(0.3)(hidden)
    
    output = Dense(classes, activation = "softmax")(hidden)
    
    model = Model(inputs=input_layer, outputs=output)
    
    return model

optimizer = Adam(learning_rate=initial_learningrate, beta_1=0.9, beta_2=0.999, amsgrad=False)

The main change to this model, other than the amount of layers, is batch normalization.

From what I have read and, let's face it, watched on YouTube, the reason why this is done is to keep the data on a similar scale and prevent gradient explosions. Apparently, non-normalised data can decrease training speed. It helps to stop the weights of the model becoming inbalanced?

here is a usefull if you're interested https://www.youtube.com/watch?v=dXB-KQYkzNU

One other thing I noticed about this model is that the dropout is smaller than I have seen in others. My guess is that becuase there are more layers the dropout is smaller in each layer, but overall it is similar? The combined number in this model is 0.75



In [ ]:


# Compile the model


model = build_model1(input_shape=(28, 28, 1), classes = 10)


model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
datagen.fit(X_train)
datagen.fit(X_val)

callbacks = [checkpoint,learning_rate_reduction]
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size ),
                              epochs = epochs,
                              validation_data=valid_datagen.flow(X_val,y_val),
                              verbose = 1,
                            callbacks = callbacks)

# On the first attempt I forgot to add the 'learning_rate_reduction'


### Keeping score

In this area I am going t keep rack of the scores I got for various models over ten epochs.
The way of explaining the model is as in https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist:

Basically model 1 means convolutional layer with 32 filters of size (3,3), then batch normalization, then dropout, then a pooling layer, then dense layer with 100 neurons, the 10 for out put. and finally the 'Accuracy and Val accuracy

- model 1: 32C3 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9141,0.9522]
- model 2: 32C5 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9378,0.9622]
- model 3: 64C3 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9196, 0.9552]
- model 4: 64C5 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9404, 0.9692]
- model 5: 128C3 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9177, 0.9532]
- model 6: 128C5 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9401, 0.9700] 

it looks like model 4 is the winner lets take this one forwards and develop it, and a 5x5 is better than 3x3 in this case. but what of 7x7?

- model 7: 64C7 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9476, 0.9682]

Interesting 9x9?
- model 8: 64C9 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9506, 0.9708]


ummm...11x11

- model 9: 64C11 - B - P2 - D(0.2)- 100 - D(0.2) - 10 [0.9516, 0.9733]
wasn't much change after this
- model 10: 64C11 - B - P2  - 32C9 - D(0.2)- 100 - D(0.2) - 10 [0.9788, 0.9802]

- model 11: 64C11 - B - P2  - 32C9 - B - P2 16C5 - D(0.2)- 100 - D(0.2) - 10 [0.9819, 0.9827]
- model 12: 64C11 - B - P2  - 32C9 - B - P2 32C5 - D(0.2)- 100 - D(0.2) - 10 [0.9834, 0.9817]

From this point I suspect we are squeezing the last few % from the model and the differences over 10 epochs may not be enough to discrimate so I am goin to switch to 20

- model 13: 64C11 - B - P2  - 32C9 - B - P2 32C5 - D(0.2)- 100 - D(0.2) - 10 [0.9870, 0.9882]. I like how similar the validation accuracy is to the accuracy. I will check how this compares to the eaderboard at this point. On the leaderboard this achieved 0.9770

- model 14: 64C11 - B - P2  - 32C9 - B - P2 65C5 - D(0.2)- 250 - D(0.2) - 10 [0.9862, 9870]

- model 15: 64C11 - B - P2  - 32C9 - B - P2 32C5 - D(0.2) - B - P2 32C3 - D(0.2) - 250 - D(0.2) - 10 [0.9870, 0.9882]
 
formula [(W−K+2P)/S]+1 for calculating image size after convolution

    W is the input volume 
    K is the Kernel size 
    P is the padding 
    S is the stride 



In [ ]:
model.load_weights('bestweights.md5')
model.summary()

<a name="section-four"></a>
## How did we do?

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best')

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best')


In [ ]:
import seaborn as sns

# used the code from https://www.kaggle.com/shahules/indian-way-to-learn-cnn to create this

y_pre_test=model.predict(X_val)
y_pre_test=np.argmax(y_pre_test,axis=1)
y_test=np.argmax(y_val,axis=1)

conf=confusion_matrix(y_test,y_pre_test)
conf=pd.DataFrame(conf,index=range(0,10),columns=range(0,10))




In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(conf, annot=True)

In [ ]:
print('out of {} samples, we got {} incorrect'.format(len(X_train), round(len(X_train) - history.history['accuracy'][-1] * len(X_train))))

In [ ]:
#code taken from here

# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_val,axis = 1) 

# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 3
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize = (10,10))
    fig.tight_layout()
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors[-18:-9]

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)



Now to make our predictions and submit!

In [ ]:
predictions = model.predict(test)

#### This made me stumble when I changed

With sequential there is a function to get the classes, but this is not true with the functional API. You get an array with the models probabilities for each class. Therefore you have to take the highest value from the array and get the corresponding class.

In [ ]:
plt.imshow(test[0].reshape(28,28))

In [ ]:
predictions[0] # looks like the 4 element is what the model thinks it is and it is over 99.99 % sure

In [ ]:

predictions = predictions.argmax(axis = -1)
predictions

In [ ]:
submission = pd.DataFrame({ 'id': Id,
                            'label': predictions })
submission.to_csv(path_or_buf ="submission.csv", index=False)

In [ ]:
#keras.backend.clear_session()